# Egyptian Universal Health Insurance - Actuarial Study
## Phase 2 Governorates Analysis (2026)

### Objective
Assess financial sustainability with a **governorate-level breakdown** for Phase 2 expansion:
1. Damietta
2. Matrouh
3. Kafr El-Sheikh
4. Minya
5. North Sinai

### Methodology
The study calculates revenues (contributions) vs. expenditures (medical costs) for each governorate individually, then aggregates for the national view.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

BASE_YEAR = 2026
PROJECTION_YEARS = 15
print(f'Study Period: {BASE_YEAR} - {BASE_YEAR + PROJECTION_YEARS}')

---
## Phase 1: Governorate-Level Data Collection

**Decision Points:**
- Specific demographics for each governorate
- Different wage levels (Urban vs. Rural wages)
- Poverty rates ('Unable-to-pay' %) varying by location

In [ ]:
# ACTUARY DECISION: Detailed Data per Governorate
# Note: Minya has higher 'unable_to_pay' ratio due to poverty rates.
# Damietta has higher 'self_employed' due to furniture industry.

GOVERNORATE_DATA = {
    'Damietta': {
        'population': 1_600_000,
        'avg_wage': 6500,
        'distribution': {'employees': 0.30, 'spouse': 0.15, 'children': 0.30, 'self_employed': 0.15, 'pensioners': 0.05, 'unable': 0.05}
    },
    'Matrouh': {
        'population': 500_000,
        'avg_wage': 5500,
        'distribution': {'employees': 0.25, 'spouse': 0.15, 'children': 0.35, 'self_employed': 0.10, 'pensioners': 0.05, 'unable': 0.10}
    },
    'Kafr_El_Sheikh': {
        'population': 3_800_000,
        'avg_wage': 5000,
        'distribution': {'employees': 0.25, 'spouse': 0.20, 'children': 0.30, 'self_employed': 0.15, 'pensioners': 0.05, 'unable': 0.05}
    },
    'Minya': {
        'population': 6_200_000,
        'avg_wage': 4500,
        'distribution': {'employees': 0.20, 'spouse': 0.15, 'children': 0.25, 'self_employed': 0.10, 'pensioners': 0.05, 'unable': 0.25} # High unable ratio
    },
    'North_Sinai': {
        'population': 480_000,
        'avg_wage': 5200,
        'distribution': {'employees': 0.25, 'spouse': 0.15, 'children': 0.30, 'self_employed': 0.10, 'pensioners': 0.05, 'unable': 0.15}
    }
}

CONTRIBUTION_RATES = {
    'employees':     {'ind': 0.01, 'emp': 0.04},
    'spouse':        {'ind': 0.03, 'emp': 0.00},
    'children':      {'ind': 0.01, 'emp': 0.00}, # Max 2 children rule simplified here
    'self_employed': {'ind': 0.04, 'emp': 0.00},
    'pensioners':    {'ind': 0.01, 'emp': 0.02}, # Employer share paid by Social Insurance
    'unable':        {'ind': 0.00, 'emp': 0.05}  # Employer share paid by State Treasury
}

# Process Data to get Counts
processed_data = []
for gov, data in GOVERNORATE_DATA.items():
    pop = data['population']
    for cat, ratio in data['distribution'].items():
        processed_data.append({
            'Governorate': gov,
            'Category': cat,
            'Count': int(pop * ratio),
            'Avg_Wage': data['avg_wage'],
            'Ind_Rate': CONTRIBUTION_RATES[cat]['ind'],
            'Emp_Rate': CONTRIBUTION_RATES[cat]['emp']
        })

df_census = pd.DataFrame(processed_data)
print('Census Data by Governorate (Sample):')
df_census.head(10)

---
## Phase 2: Assumptions & Service Costs
Using Version 6 Pricing (2026)

In [ ]:
@dataclass
class ActuarialAssumptions:
    medical_inflation: float = 0.18
    wage_growth: float = 0.10
    collection_efficiency: float = 0.85
    utilization_rates: Dict = None

SERVICE_COSTS = {
    'outpatient': 250,
    'inpatient': 3500,
    'chronic': 800,
    'surgery': 15000  # Weighted average
}

assumptions = ActuarialAssumptions(
    utilization_rates={'outpatient': 4.5, 'inpatient': 0.08, 'chronic': 0.22, 'surgery': 0.04}
)

---
## Phase 3: Governorate-Level Calculations

Calculating Net Result per Governorate:
1. **Local Revenue:** Contributions collected from residents/companies.
2. **State Subsidies:** Treasury payments for 'unable-to-pay' residents.
3. **Local Expenditure:** Value of medical services consumed.

In [ ]:
def calculate_governorate_financials(df_census, assumptions, costs):
    results = []
    
    for gov in df_census['Governorate'].unique():
        gov_df = df_census[df_census['Governorate'] == gov]
        total_pop = gov_df['Count'].sum()
        
        # 1. Revenue Calculation
        revenue = 0
        treasury_subsidy = 0
        
        for _, row in gov_df.iterrows():
            # Monthly Wage * 12 * (Ind% + Emp%) * Efficiency
            contribution = row['Count'] * row['Avg_Wage'] * 12 * (row['Ind_Rate'] + row['Emp_Rate']) * assumptions.collection_efficiency
            revenue += contribution
            
            # Track Treasury portion separately (Emp share for 'unable')
            if row['Category'] == 'unable':
                subsidy = row['Count'] * row['Avg_Wage'] * 12 * row['Emp_Rate']
                treasury_subsidy += subsidy
        
        # 2. Expenditure Calculation
        # Population * Rate * Cost
        exp_outpatient = total_pop * assumptions.utilization_rates['outpatient'] * costs['outpatient']
        exp_inpatient = total_pop * assumptions.utilization_rates['inpatient'] * 5 * costs['inpatient'] # 5 days avg
        exp_chronic = total_pop * assumptions.utilization_rates['chronic'] * 12 * costs['chronic']
        exp_surgery = total_pop * assumptions.utilization_rates['surgery'] * costs['surgery']
        
        total_exp = exp_outpatient + exp_inpatient + exp_chronic + exp_surgery
        total_exp *= 1.08 # Add 8% Admin Overhead
        
        results.append({
            'Governorate': gov,
            'Population': total_pop,
            'Revenue_Bn': revenue / 1e9,
            'Treasury_Support_Bn': treasury_subsidy / 1e9,
            'Expenditure_Bn': total_exp / 1e9,
            'Net_Surplus_Bn': (revenue - total_exp) / 1e9,
            'Cost_Coverage_Ratio': (revenue / total_exp) * 100
        })
        
    return pd.DataFrame(results)

gov_financials = calculate_governorate_financials(df_census, assumptions, SERVICE_COSTS)
print('Financial Performance by Governorate (2026):')
gov_financials.round(2)

In [ ]:
# Visualization: Revenue vs Expenditure by Governorate
plt.figure(figsize=(12, 6))

x = np.arange(len(gov_financials))
width = 0.35

plt.bar(x - width/2, gov_financials['Revenue_Bn'], width, label='Revenue (Contributions)', color='green')
plt.bar(x + width/2, gov_financials['Expenditure_Bn'], width, label='Expenditure (Services)', color='red')

plt.xlabel('Governorate')
plt.ylabel('Billion EGP')
plt.title('2026 Financial Projection by Governorate (Phase 2)')
plt.xticks(x, gov_financials['Governorate'])
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Add Treasury Support Label
for i, val in enumerate(gov_financials['Treasury_Support_Bn']):
    plt.text(i - width/2, gov_financials['Revenue_Bn'][i] + 0.1, 
             f'Subsidy:\n{val:.1f}B', ha='center', fontsize=8)

plt.tight_layout()
plt.savefig('governorate_analysis.png', dpi=150)
plt.show()

---
## Phase 4: Gap Analysis Findings

**Observation:**
- **Damietta:** Likely surplus due to higher wages and employment.
- **Minya:** Large deficit expected due to population size and high 'unable-to-pay' ratio, requiring significant State Treasury intervention.
- **North Sinai:** Special consideration for restructuring costs (not fully captured in service costs).

**Actuary Recommendation:**
Cross-subsidization is essential. Surplus from wealthy governorates (and supplementary sources like cigarette taxes) must cover deficits in Upper Egypt (Minya).


In [ ]:
# Export Governorate Report
gov_financials.to_csv('governorate_financials_2026.csv', index=False)
print('Detailed governorate report saved: governorate_financials_2026.csv')